In [58]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime as dt
from IPython.core.display import display, HTML
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 200
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
# Reading in FCC wisp data
filepath = "../data/bb_data.csv"
wisp_df = pd.read_csv(filepath)

# Reading in population densities
filepath = "../data/us2016.csv"
pop_df = pd.read_csv(filepath)

In [312]:
# Pennsylvania population density
pa_pop_df = pop_df[pop_df['stateabbr'] == 'PA']

'''
Population information:

hu = housing units
hh = households
pop = population

'''
cols = ['stateabbr','block_fips','hu2016','hh2016','pop2016']
pa_pop_df = pa_pop_df[cols]

In [333]:
def filter_wisps(raw_df):
    start_t = dt.datetime.now()
    max_up = 5
    max_down = 25
    business = 1
    state = 'PA'
    raw_df = raw_df[raw_df['Max Advertised Upstream Speed (mbps)'] >= max_up]
    raw_df = raw_df[raw_df['Max Advertised Downstream Speed (mbps)'] >= max_down]
    raw_df = raw_df[raw_df['Business'] == business]
    raw_df = raw_df[raw_df['State'] == state]
    final_t = dt.datetime.now()
    print(f'took {final_t - start_t} to run!')
    return raw_df

In [334]:
pa_wisp_df = filter_wisps(wisp_df)
pa_wisp_df.shape

took 0:00:36.156326 to run!


(489119, 17)

In [313]:
'''
Create Montgomery county subsets

PA state FIPS code 42
Montgomery county FIPS code 091
'''
pa_pop_df['block_fips']= pa_pop_df['block_fips'].apply(str)
pa_pop_df['mont_county'] = pa_pop_df['block_fips'].map(lambda x: True if x[:5] == '42091' else False)
mont_pop_df = pa_pop_df[pa_pop_df['mont_county']==True]

pa_wisp_df['Census Block FIPS Code'] = pa_wisp_df['Census Block FIPS Code'].apply(str)
pa_wisp_df['mont_county'] = pa_wisp_df['Census Block FIPS Code'].map(lambda x: True if x[:5] == '42091' else False)
mont_wisp_df = pa_wisp_df[pa_wisp_df['mont_county'] == True]

In [314]:
# Using the groubpy function to get metrics for each census block: count, types, mean up/down speeds
def get_metrics(raw_df):
    
    # A count of all the unique WISP's in a given census block,  if block isn't included in dataframe, then there are no WISPs in that block
    wisp_count_df = pd.DataFrame({'wisp_count':raw_df.groupby(['Census Block FIPS Code']).size()}).reset_index()
    wisp_count_df.set_index('Census Block FIPS Code', inplace=True)

    # Number of WISPs allowed to serve consumers in the given block
    consumer_df = pd.DataFrame({'consumer_num':raw_df.groupby(['Census Block FIPS Code','Consumer']).size()}).reset_index()
    consumer_df = consumer_df[consumer_df['Consumer']==1]
    consumer_df.set_index('Census Block FIPS Code', inplace=True)

    # Number of WISPs allowed to serve businesses in the given block
    business_df = pd.DataFrame({'business_num':raw_df.groupby(['Census Block FIPS Code', 'Business']).size()}).reset_index()
    business_df = business_df[business_df['Business']==1]
    business_df.set_index('Census Block FIPS Code', inplace = True)

    # Average up/downstream speeds in each census block
    adv_max_down_df = pd.DataFrame({'adv_max_down':raw_df.groupby(['Census Block FIPS Code'])['Max Advertised Downstream Speed (mbps)'].mean()}).reset_index()
    adv_max_down_df.set_index('Census Block FIPS Code', inplace = True)
    
    adv_max_up_df = pd.DataFrame({'adv_max_up':raw_df.groupby(['Census Block FIPS Code'])['Max Advertised Upstream Speed (mbps)'].mean()}).reset_index()
    adv_max_up_df.set_index('Census Block FIPS Code', inplace = True)
    
    cir_max_up_df = pd.DataFrame({'cir_max_up':raw_df.groupby(['Census Block FIPS Code'])['Max CIR Upstream Speed (mbps)'].mean()}).reset_index()
    cir_max_up_df.set_index('Census Block FIPS Code', inplace = True)
    
    cir_max_down_df = pd.DataFrame({'cir_max_down':raw_df.groupby(['Census Block FIPS Code'])['Max CIR Downstream Speed (mbps)'].mean()}).reset_index()
    cir_max_down_df.set_index('Census Block FIPS Code', inplace = True)
    
    #TODO: Merge the 7 df's
    df_list = [wisp_count_df,consumer_df,business_df,adv_max_down_df,adv_max_up_df,cir_max_up_df,cir_max_down_df]
    print('Merging dataframes...')
    for i in df_list:
        if wisp_count_df.equals(i):
            final_df = wisp_count_df
        else:
            final_df = pd.merge(final_df,i,how='outer',left_index=True, right_index=True)
    print('Done merging dataframes!')
    final_df.drop(['Consumer', 'Business'], axis = 1, inplace = True)
    return final_df 

In [315]:
# Get important metrics for montgomery county
mont_metrics_df = get_metrics(mont_wisp_df)

# Get metrics for whole state of PA
pa_metrics_df = get_metrics(pa_wisp_df)

Merging dataframes...
Done merging dataframes!
Merging dataframes...
Done merging dataframes!


In [316]:
# Merging population data and WISP metrics
def merge_pop_wisp(pop_df,wisp_df):
    if (pop_df.index.name != 'block_fips'):
        pop_df.set_index('block_fips',inplace = True)
    final_df = pd.merge(pop_df, wisp_df,how = 'left', left_index = True, right_index = True)
    return final_df 

In [338]:
county_df = merge_pop_wisp(mont_pop_df, mont_metrics_df)
county_df.fillna(value = 0, inplace = True)
county_df['density'] = county_df['wisp_count']/(county_df['pop2016'] + 1)
county_df.reset_index(inplace = True)

In [340]:
state_df = merge_pop_wisp(pa_pop_df, pa_metrics_df)
state_df.fillna(value = 0, inplace = True)
state_df['density'] = state_df['wisp_count']/(state_df['pop2016'] + 1)
state_df.reset_index(inplace = True)

In [341]:
# Write county and state data to csv
county_df.to_csv('../data/montgomery_wisp_metrics.txt')
state_df.to_csv('../data/pa_wisp_metrics.txt')

## Baseline statistics

What are the important measures we want to see?

Breakdown of # wisps
Density map (wisp/population) in a given block
Heatmap of number of wisps operating in given census block (filter by type, up/down speed)
Correlation between WISP/population

In [344]:
county_df['wisp_count'].value_counts()

2.0    9682
1.0    2171
0.0    1909
3.0       7
Name: wisp_count, dtype: int64